![figure](lab7/politecnico_h-01.png)

# **Eletrónica Programável**
#### Licenciatura em Engenharia Eletrotécnica e de Computadores

## Trabalho Laboratorial 7 - Controlador VGA

-----------------------


## Introdução ##
Neste trabalho propomos a implementação faseada de um jogo de pong usando controladores VGA baseados nos esquemas object-mapped e tile-mapped. Este documento não fornece detalhes de utilização do Vivado. Sempre que tiver dúvidas a este nível, consulte o enunciado do Lab1.


### Objetivos ###

Neste trabalho vamos testar sequencialmente:
* Varrimento do monitor VGA.
* Esquema object-mapped.
* Esquema object-mapped animado.
* Esquema tile-mapped para texto.
* Jogo completo do PONG

Nas instruções abaixo **{sources}** refere-se a `C:\Xilinx\EEC_EP\sources` e **{labs}** refere-se a `(C:\Xilinx\EEC_EP\labs)`


_______________________

## Passo 1 - Interface VGA estático ##

### Passo 1.1 - Varrimento do monitor VGA ### 

Para fazer o varimento de um monitor VGA,é necessário implementar um módulo de sincronização que gere os sinais de varrimento horizontal e vertical. O código VHDL deste circuito é fornecido no ficheiro **vga_sync.vhd**. Para verificar se o circuito funciona como esperado, a entrada rgb deste circuito é conectada aos interruptores da Basys3. Os interruptores devem permitir definir a cor do ecrã, de entre as 4096 combinações possíveis (são 12 bits de cor). O código VHDL deste circuito de teste é fornecido no ficheiro **vga_sync_test.vhd**. 


<div>
<img src="lab7/fig1.png" width=700"/>
</div>
                                

* Comece por descarregar os arquivos fonte fornecidos (Github ou Moodle) e salve-os em `{sources}/lab7`.


* Abra o Vivado a partir do icon no ambinete de trabalho ou: `Start > All Programs > Xilinx Design Tools > Vivado 2022.1`. 


* Selecione **Create Project** para começar o wizard. Na caixa de diálogo *Create A New Vivado Project*, selecione **Next**.


* Clique no botão *Procurar* do campo de localização do projeto do novo formulário de projeto, navegue até `c:/Xilinx/EEC_EP/labs` e clique em **Select**. Digite *lab7* no campo de nome do projeto. Certifique-se de que a caixa *Create Project Subdirectory* esteja marcada. Clique em **Next**.


* Selecione a opção **RTL Project** no formulário *Project Type* e clique em **Next**. Certifique-se de que a caixa *Do not specify sources at this time* está marcada.


* No formulário **Default Part**, selecione a opção *Boards* e escolha a placa **Basys3**. Clique em **Next** e depois em **Finish** para criar o projeto Vivado.


* No painel *Sources*, selecione **Add Sources** e siga os passos necessários (*Add or create design sources*) para adicionar todos os ficheiros VHDL necessários (descritos acima).


* Abra os ficheiros no Vivado, observe o código e verifique que eles correspondem aos módulos representados na figura acima. 


* O ficheiro de topo (**vga_sync_test.vhd**) instancia um módulo (**clk_wiz_0**) que não existe no projeto. Trata-se de um módulo de gestão de sinais de relógio, que gera um sinal de 50MHz a partir da entrada de 100MHz. Para adicioná-lo ao projeto, escolha a opção **IP Catalog** na janela do *Flow Navigator*, como mostra a figura e faça duplo clique no **Clocking Wizard*.


<div>
<img src="lab7/fig2.png" width=700"/>
</div>


* Na janela de configuração do *Clocking Wizard*, deixe o nome por omissão (clk_wiz_0) e:
    * No separador **Clocking Options**, escolha a primitiva PLL.
    * No separador **Output Clocks**, configure a *Requested Output Freq (MHz)* para 50.
    * Faça **OK**.


* Note agora que o módulo já aparece correctamente instanciado na hierarquia do módulo **vga_test**.


* Adicione agora o ficheiro de restrições, defina os pinos das entradas/saídas e implemente o circuito na Basys3.


* Teste o circuito ligando um monitor VGA à placa Basys3. Observe as diferentes cores no ecrã criadas pelas várias combinações de entrada nos interruptores.



### Passo 1.2 - Esquema object-mapped estático ### 

Vamos exemplificar a geração de objectos num monitor VGA, começando por criar um ambiente com objectos rectangulares fixos. Cada objecto é descrito pelas suas coordenadas. Por exemplo, a parede (wall) da figura pode ser definida com 4 pixéis de largura entre as colunas 32 e 35. As coordenadas são então WALL_X_L=32 e WALL_X_R=35. É necessário ainda um sinal que define a cor da parede (wall_rgb) e outro que indica que o objecto deve ser enviado para o ecrã (wall_on). Este é asserido sempre que o varrimento horizontal esteja na região da parede

 

<div>
<img src="lab7/fig3.png" width=800"/>
</div>
            
            
Para a barra (padle) temos algo semelhante, mas agora temos também de incluir fronteiras no eixo do Y. A barra deve ter um comprimento de 72 pixéis (definido em BAR_Y_SIZE) e deve estar a meio do ecrã, pelo que a coordenada do topo da barra é metade do valor máximo de Y menos metade do comprimento da barra (480/2 - 72/2). A coordenada da base é igual à coordenada do topo mais o comprimento da barra. O código da bola pode ser feito da mesma forma. 


<div>
<img src="lab7/fig4.png" width=500"/>
</div>
                                   

Finalmente um multiplexador que examina os sinais on de cada objecto e envia para o monitor o sinal **RGB** correspondente. Combinando este circuito com o anterior, podemos então visualizar os objectos no ecrã. 


* Adicione ao projeto os ficheiros que permitem gerr estes objectos de forma estática no ecrã (**pong_graph_st.vhd.** e **pong_top_st.vhd**).


* Abra os ficheiros e analise o código. Observe como são geradas as cooredenadas dos objectos. 


* Defina o **pong_top_st.vhd** como sendo o módulo de topo (*Set as Top*), e altere o ficheiro de pinos de entrada/saída (com base nos sinais de entrada e saída desta entidade). 


* Implemente o circuito, programe a Basys3 e observe o resultado no ecrã. 



______

## Passo 2 - Interface VGA dinâmico ##

### Passo 2.1 - Objectos não rectangulares e animados ### 

Para implementar um objecto que não seja rectangular a melhor forma é especificar o padrão num mapa de bits e gerar os sinais **RGB** e **on** de acordo com o mapa. Por exemplo, se queremos desenhar uma bola redonda, definimos um quadrado de pixéis onde se define o objecto redondo. Esta matriz seria então armazenada numa ROM de 8 por 8 pixéis.

Para facilitar a animação de objectos definidos desta forma, os sinais que definem as fronteiras do objecto não devem ser constantes mas sinais de entrada, como se mostra na figura. É então necessário verificar se o varrimento está na zona do mapa da bola e mapear o pixel actual ao endereço da memória. 


<div>
<img src="lab7/fig5.png" width="800"/>
</div>

**Nota**: É também possível usar três ROMs distintas para armazenar valores RGB e assim gerar bolas de múltiplas cores. Estas modificações estão incluídas no ficheiro fornecido na próxima secção.


Quando um objecto muda a sua posição gradualmente, cria a ilusão de estar em movimento. Para conseguir este efeito, vamos usar registos para guardar a posição das coordenadas de fronteira e actualizar esses registos a cada novo varrimento do ecrã (60 vezes por segundo). É necessário por isso gerar um sinal de relógio a 60Hz (**refr_tick**).


* Adicione ao projeto os ficheiros **pong_graph_animate.vhd.** e **pong_top_an.vhd**. Defina este último como módulo de topo *Set as Top*. 


* Abra o ficheiro **pong_graph_animate.vhd.** e observe o código que permite gerar a bola redonda. Nas linhas 59 a 79 define-se a memória ROM. Nas linhas163 a 165 define-se que a bola deve estar **ON** (ou seja, deve ser enviada a cor da bola para o ecrã) apenas quando:
    * o varrimento está dentro do quadrado que define a bola (**sq_ball_on='1'**)
    * o bit armazenado na memória está ao nível lógico alto (**rom_bit='1'**)


* Observe o código que permite gerar o sinal **refr_tick** nas linhas 103 a 108.


A barra vai ser agora controlada por botões de pressão, para que seja deslocada para cima e para baixo. As constantes que definem as coordenadas de fronteira devem então ser alteradas para sinais (**bar_y_t** para o topo e **bar_y_b** para o fundo) e deve ser criado um registo (**bar_y_reg**) que armazene a coordenada actual da fronteira de topo. Sempre que os botões exteriores são pressionados (btn(1) e btn(0) para andar para cima e para baixo, respectivamente), o valor neste registo deve ser incrementado ou decrementado, à frequência de 60Hz. 


* Observe a diferença entre o código que define a parede estática (linhas 109 a 118) com o código que define a barra dinâmica (linhas 119 a 144).


* Note que o valor do incremento é definido pela constante **BAR_V** (linhas 137 e 139). A magnitude deste incremento vai definir a velocidade que queremos imprimir à barra. A barra deve parar quando atinge o topo ou o fundo do ecrã.


A animação da bola é um pouco mais complexa. Neste caso são necessários dois registos, um para a posição horizontal e outro para a posição vertical (**ball_x_reg** e **ball_y_reg**). Por outro lado é necessário decompor a velocidade numa componente no eixo do **X** e outra no eixo do **Y**, para permitir que a bola mude de direcção ao bater na barra ou nos limites do ecrã. Estas componentes podem assumir valores positivos ou negativos, definidos pelas constantes **BALL_V_P** e **BALL_V_N**, respectivamente. 


* Observe o código da animação da bola (linhas 144 a 194). O movimento é imprimido à bola rectangular, da mesma forma que é feito com a barra, com a diferença que agora temos movimento em ambos os eixos fazendo uso dos registos **ball_x_reg** e **ball_y_reg**, atualizados nas linhas 169 a 174.


* A velocidade da bola em vada eixo (**x_delta_reg** e **y_delta_reg**) vai depender da direção que queremos dar à bola em cada ressalto. Obseve o código nas linhas 176 a 194.


* Implemente este circuito na Basys 3. Lembre-se que é necessário atualizar o ficheiro de pinos de entrada/saída. 


* Programe a placa e use os botões de pressão para movimentar a barra. 



______________

## Passo 3 - Esquema tile-mapped para texto. ##

### Passo 3.1 ### 

Para poder representar texto num monitor VGA é necessário primeiro construir uma memória com os caracteres fonte, tal como é feito no ficheiro **font_rom.vhd**. 


* Adicione este ficheiro ao projecto. Abra-o e observe o código. Note que se trata de uma memória ROM (definida como uma constante), que permite armazenar dados de 8 bits (DATA_WIDTH: integer:=8) e que tem 2048 posições de memória (ADDR_WIDTH: integer:=11). Cara caractere **ASCII** ocupa 16 posições de memória 


Para verificar o funcionamento da ROM vamos usar um circuito que permita visualizar todos os caracteres no ecrã. O código deste circuito é fornecido no ficheiro **font_test_gen.vhd**. 
O ficheiro **font_test_top.vhd** combina estes circuitos com o circuito de sincronismo do ecrã e gerador de relógio, permitindo implementar o circuito de teste na placa de desenvolvimento. 

* Adicione estes ficheiros ao projecto. Defina o circuito **font_test_top.vhd** como sendo o módulo de topo (*Set as Top*) e modifique o ficheiro de pinos de entrada/saída. Teste o circuito e observe os caracteres no ecrã. 




_________

## Desafio ##

### Desafio 1 - Modo texto ###

Altere o código no ficheiro **font_test_gen.vhd**  para aumentar o tamanho de letra. Veja como na secção 13.2.5 (pag. 297) do livro de referência:

<div>
<img src="lab7/fig9.png" width="500"/>
</div>

### Desafio 2 - Modo animate ###

Altere o código no ficheiro **pong_graph_animate.vhd**  para aumentar o tamanho da bola em 4 vezes.

### Desafio 3 - Pong Game ###

Crie um novo projecto de nome PONG e adicione todas as fontes fornecidas na página da UC. Note que vai ter de adicionar aqui também um módulo **Clocking Wizard** para obter um sinal de 50MHz, tal como vez no passo 1.1 deste trabalho. Programe a placa e divirta-se!
